In [1]:
import json
import numpy as np
from bias_explorer.operations import report
from bias_explorer.utils import system, dataloader
import glob
import os
import pandas as pd

In [2]:
report_path = './data/fairface/reports/openCLIP/ViT-L-14/laion400m_e31/age_race_gender/accuracy_report.csv'
df = dataloader.load_df(report_path)

In [3]:
best = df.loc[df['accuracy'].idxmax()]
best_mode = best['Mode']
best_acc = best['accuracy']
print(best_mode)
print(best_acc)

Top 10
0.9601


In [4]:
def collect_reports(ds_path, ln, metric):
    ds_list = os.listdir(ds_path)
    reports = {}
    for dsource in ds_list:
        report_path = f"{ds_path}/{dsource}/{ln}/{metric}_report.csv"
        ds_df = dataloader.load_df(report_path)
        ds_df = ds_df.drop(columns=['Unnamed: 0'])
        reports[dsource] = ds_df
    return reports

In [5]:
conf = dataloader.load_config()
model = conf['Model']
backbone = conf['Backbone']
report_path = conf['Reports']
metric = conf['Metric']
ln = system.grab_label_name(conf['Labels'])
ds_path = f"{report_path}/{model}/{backbone}"


In [6]:
reps = collect_reports(ds_path, ln, metric)

In [8]:
filtered_reports = {}
for dsname, df in reps.items():
    filtered_reports[dsname] = df.loc[df['accuracy'].idxmax()]

In [16]:
for i in range(1, 10):
    print(i)

1
2
3
4
5
6
7
8
9
